[Interactive Version with SageMathCell](https://olgabelitskaya.github.io/MLE_ND_P3_SMC.html)
# Python Modules & Helpful Functions

In [ ]:
import warnings; warnings.filterwarnings("ignore")
from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings("ignore",category=DataConversionWarning)
import numpy,pandas,pylab,seaborn,time,os
pylab.style.use('seaborn-pastel')
path='../input/wholesale-customers-data-set/'
file='Wholesale customers data.csv'
cmap=seaborn.cubehelix_palette(2,start=.1,rot=-.25,as_cmap=True)
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score
os.listdir('../input')

In [ ]:
# https://github.com/udacity/machine-learning/blob/master/projects/customer_segments/visuals.py
def pca_results(good_data,pca):
    dimensions=['Dimension {}'.format(i) 
                for i in range(1,len(pca.components_)+1)]
    components=pandas.DataFrame(numpy.round(pca.components_,4),
                                columns=good_data.keys())
    components.index=dimensions
    ratios=pca.explained_variance_ratio_.reshape(len(pca.components_),1)
    variance_ratios=pandas.DataFrame(numpy.round(ratios,4),
                                     columns=['Explained Variance'])
    variance_ratios.index=dimensions 
    fig,ax=pylab.subplots(figsize=(11,5))
    components.plot(ax=ax,kind='bar');
    ax.set_ylabel("Feature Weights",fontsize=12)
    ax.set_xticklabels(dimensions,rotation=0)
    for i,ev in enumerate(pca.explained_variance_ratio_):
        ax.text(i-0.40,ax.get_ylim()[1]+0.05,
                "Explained Variance \n %.4f"%(ev))
    pylab.grid(); pylab.show()
    return pandas.concat([variance_ratios,components],axis=int(1))
def cluster_results(reduced_data,preds,centers,pca_samples,alg):
    predictions=pandas.DataFrame(preds,columns=['Cluster'])
    plot_data=pandas.concat([predictions,reduced_data],axis=int(1))
    fig,ax=pylab.subplots(figsize=(11,7))
    for i,cluster in plot_data.groupby('Cluster'):   
        cluster.plot(ax=ax,kind='scatter',
                     x='Dimension 1',y='Dimension 2',
                     alpha=0.8,edgecolors='slategray',
                     c=cmap((i)*1.0/(len(centers)-1)),
                     label='Cluster %i'%(i),s=30);
    for i,c in enumerate(centers):
        ax.scatter(x=c[0],y=c[1],color='white',edgecolors='black',
                   alpha=1,linewidth=2,marker='o',s=250);
        ax.scatter(x=c[0],y=c[1],marker='$%d$'%(i),
                   alpha=1,s=120,color='black');
        ax.scatter(x=pca_samples[:,0],y=pca_samples[:,1],s=150,
                   linewidth=4,color='#ff5a8b',marker='x');
    ti="%s Cluster Learning on PCA-Reduced Data\n"+\
       "Centroids Marked by Numbers\n"+\
       "Transformed Sample Data Marked by Red Crosses"
    ax.set_title(ti%alg,fontsize=15); pylab.show()
def biplot(good_data,reduced_data,pca):
    fig,ax=pylab.subplots(figsize=(11,7))
    x=reduced_data.loc[:,'Dimension 1']
    y=reduced_data.loc[:,'Dimension 2']
    ax.scatter(x=x,y=y,facecolors='slategray',
               edgecolors=cmap((x*y)**2),s=70,alpha=.4)
    feature_vectors=pca.components_.T
    arrow_size,text_pos=7.,5.
    for i,v in enumerate(feature_vectors):
        ax.arrow(0,0,arrow_size*v[0],arrow_size*v[1],
                 head_width=.2,head_length=.2,
                 linewidth=2,color='#ff5a8b')
        ax.text(v[0]*text_pos,v[1]*text_pos,
                good_data.columns[i],color='black',
                ha='center',va='center',fontsize=12)
    ax.set_xlabel("Dimension 1",fontsize=10)
    ax.set_ylabel("Dimension 2",fontsize=10)
    ax.set_title("PC plane with original feature projections.",
                 fontsize=15); pylab.show()
def channel_results(reduced_data,outliers,pca_samples):
    full_data=pandas.read_csv(path+file)
    channel=pandas.DataFrame(full_data['Channel'],columns=['Channel'])
    channel=channel.drop(channel.index[outliers]).reset_index(drop=True)
    labeled=pandas.concat([reduced_data,channel],axis=int(1))
    fig,ax=pylab.subplots(figsize=(11,5))
    labels=['Hotel/Restaurant/Cafe','Retailer']
    grouped=labeled.groupby('Channel')
    for i,channel in grouped:   
        channel.plot(ax=ax,kind='scatter',
                     x='Dimension 1',y='Dimension 2',
                     color=cmap((i-1)*1.0/2),label=labels[i-1],s=30)  
    for i,sample in enumerate(pca_samples):
        ax.scatter(x=sample[0],y=sample[1],s=230,linewidth=3,
                   color='black',marker='o',facecolors='none');
        ax.scatter(x=sample[0]+0.25,y=sample[1]+0.3,
                   marker='$%d$'%(i),alpha=1,s=200,color='black')
    ti="PCA-Reduced Data Labeled by 'Channel'\n"+\
       "Transformed Sample Data Circled"
    ax.set_title(ti,fontsize=15); pylab.show()

In [ ]:
def split_fit():
    X_train,X_test,y_train,y_test=\
    train_test_split(new_data,target,test_size=0.25,random_state=1)
    print ("Training and testing splitting was successful.")
    regressor1=DecisionTreeRegressor(random_state=1)
    regressor1.fit(X_train,y_train)
    regressor2=RandomForestRegressor(random_state=1)
    regressor2.fit(X_train,y_train)
    score1=regressor1.score(X_test,y_test)
    score2=regressor2.score(X_test,y_test)
    st1="Decision Tree Regressor. "
    st2="Random Forest Regressor. "
    st3="The score of the prediction using the testing set is {}."
    print(st1+st3.format(score1)); print(st2+st3.format(score2))
    return regressor1,regressor2
def plot_feature_importance(f,regressor1,regressor2):
    D1=dict(zip(new_data,regressor1.feature_importances_))
    D2=dict(zip(new_data,regressor2.feature_importances_))
    pylab.figure(figsize=(11,5)); 
    pylab.subplot(121); pylab.grid()
    pylab.bar(range(len(D1)),D1.values(),alpha=.5,
              color='slategrey',align='center')
    pylab.xticks(range(len(D1)),D1.keys(),fontsize=7)
    st='Experiment with "%s": Feature Importances, \n'
    pylab.title(st%f+'Decision Tree Regressor')
    pylab.subplot(122); pylab.grid()
    pylab.bar(range(len(D2)),D2.values(),alpha=.5,
              color='slategrey',align='center')
    pylab.xticks(range(len(D2)),D2.keys(),fontsize=7)
    pylab.title(st%f+'Random Forest Regressor'); pylab.show()

# Data

In [ ]:
data=pandas.read_csv(path+file)
data.drop(['Region','Channel'],axis=1,inplace=True)
data.columns=['Fresh','Milk','Grocery',
              'Frozen','Detergents_Paper','Delicatessen']
ti="The dataset has {} samples with {} features each."
print(ti.format(*data.shape)); data.describe().T

In [ ]:
data.plot.area(stacked=False,figsize=(11,5))
pylab.grid(); pylab.show()

In [ ]:
# selecting samples
indices=[23,25,27]
def hex_to_rgb(chex):
        chex=chex.lstrip('#'); n=len(chex)
        return tuple(int(chex[i:i+n//3],16)/255. for i in range(0,n,n//3))
samples=pandas.DataFrame(data.loc[indices],
                         columns=data.keys()).reset_index(drop=True)
samples.index=['C0','C1','C2']; samples.loc['Mean']=samples.mean()
print ("Chosen samples of the wholesale customers' dataset:"); samples
cols=['#1b2c45','#5a8bbd','#008b8b','#ff5a8b']
samples.T.plot(figsize=(11,5),color=[hex_to_rgb(el) for el in cols])
pylab.grid(); pylab.show()

In [ ]:
pylab.figure(figsize=(11,5))
p_samples=samples.iloc[:].T.apply(lambda x:100.0*x/x.sum())
seaborn.heatmap(p_samples,cmap=cmap,annot=True,
                annot_kws={"color":"White","size":20},fmt='.1f')
ti="Product categories in percentages for the sample customers"
pylab.title(ti,fontsize=15)
pylab.xticks(ha='center',fontsize=12)
pylab.yticks(fontsize=10); pylab.show()

# Feature Relevance
### Experiment with Delicatessen

In [ ]:
new_data=data.drop('Delicatessen',axis=int(1))
target=data['Delicatessen']
new_data_target=pandas.concat([new_data,target],axis=int(1))
print ("The correlation table for the choosen feature 'Delicatessen'")
pearson=new_data_target.corr(method='pearson')
corr_with_delicatessen=pearson.iloc[int(-1)][:-int(1)]
corr_with_delicatessen[abs(corr_with_delicatessen).argsort()[::-int(1)]]

In [ ]:
pylab.figure(figsize=(11,5))
seaborn.distplot(target,color='#1b2c45',bins=200,
                 hist_kws={'color':'SlateGrey'})
pylab.xlabel("Delicatessen",fontsize=15)
pylab.title("Customers' Annual Spending",fontsize=12)
pylab.grid(); pylab.show()

In [ ]:
regressor1,regressor2=split_fit()
plot_feature_importance('Delicatessen',regressor1,regressor2)

### Experiment with Grocery

In [ ]:
new_data=data.drop('Grocery',axis=int(1)); target=data['Grocery']
new_data_target=pandas.concat([new_data,target],axis=int(1))
print ("The correlation table for the choosen feature 'Grocery'")
pearson=new_data_target.corr(method='pearson')
corr_with_grocery=pearson.iloc[int(-1)][:-int(1)]
corr_with_grocery[abs(corr_with_grocery).argsort()[::-int(1)]]

In [ ]:
pylab.figure(figsize=(11,5))
seaborn.distplot(target,color='#1b2c45',bins=200,
                 hist_kws={'color':'SlateGrey'})
pylab.xlabel("Grocery",fontsize=15)
pylab.title("Customers' Annual Spending",fontsize=12)
pylab.grid(); pylab.show()

In [ ]:
regressor1,regressor2=split_fit()
plot_feature_importance('Grocery',regressor1,regressor2)

### Feature Distributions

In [ ]:
axes=pandas.plotting.scatter_matrix(data,alpha=.3,figsize=(11,10),
                                    diagonal='hist',c='#1b2c45',
                                    hist_kwds={'color':'SlateGrey','bins':100})
corr=data.corr().values
for i,j in zip(*numpy.triu_indices_from(axes,k=1)):
    axes[i,j].annotate("%.3f" %corr[i,j],(.8,.8),fontsize=12,
                       xycoords='axes fraction',ha='center',va='center')
pylab.grid(); pylab.show()

In [ ]:
print ("Pearson correlation coefficients")
data.corr(method='pearson')

In [ ]:
pylab.figure(figsize=(11,5)); mu,sigma=3.,1.
s=numpy.random.lognormal(mu,sigma,1000)
count,bins,ignored=pylab.hist(s,200,align='mid',density=True,
                              alpha=.5,color='slategrey')
x=numpy.linspace(min(bins),max(bins),10000)
pdf=(numpy.exp(-(numpy.log(x)-mu)**2/(2*sigma**2))\
     /(x*sigma*numpy.sqrt(2*numpy.pi)))
pylab.plot(x,pdf,linewidth=2,color='#1b2c45')
pylab.axis('tight'); pylab.grid()
pylab.title('Example of a Random Log-Normal Distribution',
            fontsize=15); pylab.show()

In [ ]:
log_data=numpy.log(data); log_samples=numpy.log(samples)
corr=data.corr().values
axes=pandas.plotting.scatter_matrix(log_data,alpha=.3,figsize=(9,8),
                                    diagonal='hist',c='#1b2c45',
                                    hist_kwds={'color':'SlateGrey','bins':100})
for i,j in zip(*numpy.triu_indices_from(axes,k=1)):
    axes[i,j].annotate("%.3f" %corr[i,j],(.2,.2),fontsize=12,
                       xycoords='axes fraction',ha='center',va='center')
pylab.grid(); pylab.show()

In [ ]:
log_samples

In [ ]:
print ("Pearson correlation coefficients")
log_data.corr(method='pearson')

# Outlier Detection

In [ ]:
for feature in log_data.keys():
    Q1,Q3=numpy.percentile(log_data[feature],25),\
          numpy.percentile(log_data[feature],75)
    step=(Q3-Q1)*1.5
    st="Data points considered outliers for the feature '{}':"
    print(st.format(feature))
    display(log_data[~((log_data[feature]>=Q1-step)&(log_data[feature]<=Q3+step))])
outliers=[65,66,75,128,154]
print("Data outliers: '{}'".format(outliers))
good_data=log_data.drop(log_data.index[outliers]).reset_index(drop=True)

In [ ]:
out_percentiles=data.rank(pct=True).iloc[outliers]
pylab.figure(figsize=(11,5))
pylab.xticks(fontsize=10); pylab.yticks(fontsize=10)
seaborn.heatmap(out_percentiles,annot=True,cmap=cmap,
                annot_kws={'color':'White','size':'20'})
pylab.title('Multiple Outliers Heatmap',fontsize=15); pylab.show()

# Feature Transformation

In [ ]:
pca=PCA(n_components=len(good_data.keys())).fit(good_data)
pca_samples=pca.transform(log_samples) 
pca_result=pca_results(good_data,pca); pca_result

In [ ]:
pylab.figure(figsize=(11,4))
pylab.xlabel('Dimension',fontsize=10)
pylab.plot(numpy.arange(1,7),
           numpy.cumsum(pca.explained_variance_ratio_),
           '-o',c='#1b2c45')
pylab.title('Explained Variance Ratio',fontsize=15)
pylab.grid(); pylab.show()
print ("The first and second principal components explained {}%"\
.format(sum(pca_result['Explained Variance'][:int(2)])*100.0))
print ("The first four principal components explained {}%"\
.format(sum(pca_result['Explained Variance'][:int(4)])*100.0))

In [ ]:
pandas.DataFrame(numpy.round(pca_samples,4),
                 columns=pca_result.index.values,
                 index=['C0','C1','C2','Mean'])

In [ ]:
pca=PCA(n_components=2).fit(good_data)
pca_samples=pca.transform(log_samples)
reduced_data=pandas.DataFrame(pca.transform(good_data),
                              columns=['Dimension 1','Dimension 2'])
pandas.DataFrame(numpy.round(pca_samples,4),
                 columns=['Dimension 1','Dimension 2'],
                 index=['C0','C1','C2','Mean'])

In [ ]:
biplot(good_data,reduced_data,pca)

# Creating Clusters

In [ ]:
for n in list(range(2,12)):
    clusterer=KMeans(n_clusters=n).fit(reduced_data)
    preds=clusterer.predict(reduced_data)
    centers=clusterer.cluster_centers_
    sample_preds=clusterer.predict(pca_samples)
    score=silhouette_score(reduced_data,preds)
    st="For number of clusters = {}, the silhouette_score is : {}"
    print(st.format(n,score))

In [ ]:
for n in list(range(2,12)):
    clusterer2=GaussianMixture(n_components=n,
                               covariance_type='full').fit(reduced_data)
    preds2=clusterer2.predict(reduced_data); centers2=clusterer2.means_
    sample_preds2=clusterer2.predict(pca_samples)
    score2=silhouette_score(reduced_data,preds2,metric='mahalanobis')
    print("For number of clusters = {}, the silhouette_score is : {}"\
          .format(n,score2))

In [ ]:
n,k=300,6; kmeans_train_times=[]; gmm_train_times=[]
for k in range(2,k+1):
    cum_time=0.
    for i in range(n):
        start=time.time()
        KMeans(n_clusters=k).fit(reduced_data)
        cum_time+=(time.time()-start)
    kmeans_train_times.append([k,cum_time/n])
km_df=pandas.DataFrame(kmeans_train_times,
                       columns=['KM_Clusters','KM_Time'])
for k in range(2,k+1):
    cum_time=0.
    for i in range(n):
        start=time.time()
        GaussianMixture(n_components=k).fit(reduced_data)
        cum_time+=(time.time()-start)
    gmm_train_times.append([k,cum_time/n])
gmm_df=pandas.DataFrame(gmm_train_times,
                        columns=['GMM_Components','GMM_Time'])
times_df=km_df.join(gmm_df)
pylab.figure(figsize=(11,4))
pylab.plot(times_df.GMM_Components,times_df.GMM_Time*1000.,
           '-o',c='#5a8bbd',label='GMM Train Time')
pylab.plot(times_df.GMM_Components,times_df.KM_Time*1000.,
           '-o',c='#008b8b',label='Kmeans Train Time')
pylab.ylabel('Train time (in millisec.)',fontsize=10)
pylab.xlabel('Cluster/Components Used in Training',fontsize=10)
st='Training Time for Different Cluster/Component Sizes\n'+\
   'Averaged Over {} Runs Per Size'
pylab.title(st.format(n),fontsize=15)
pylab.legend(loc='best'); pylab.grid(); pylab.show()

In [ ]:
clusterer=KMeans(n_clusters=2).fit(reduced_data)
sample_preds=clusterer.predict(pca_samples)
preds=clusterer.predict(reduced_data)
centers=clusterer.cluster_centers_
score=silhouette_score(reduced_data,preds)
cluster_results(reduced_data,preds,centers,pca_samples,'K-Means')

In [ ]:
clusterer2=GaussianMixture(n_components=2,
                           covariance_type='full').fit(reduced_data)
sample_preds2=clusterer2.predict(pca_samples)
preds2=clusterer2.predict(reduced_data)
centers2=clusterer2.means_; score2=silhouette_score(reduced_data,preds2)
cluster_results(reduced_data,preds2,centers2,
                pca_samples,'Gaussian Mixture Model')

In [ ]:
print ("Data Means")
pandas.DataFrame(numpy.mean(data)).T

In [ ]:
log_centers=pca.inverse_transform(centers)
true_centers=numpy.exp(log_centers)
segments=['Segment {}'.format(i) for i in range(0,len(centers))]
true_centers=pandas.DataFrame(numpy.round(true_centers),
                              columns=data.keys(),index=segments)
print ("K-Means True Centers"); true_centers

In [ ]:
true_centers=true_centers.append(data.describe().loc['mean'])
true_centers.plot(kind='bar',figsize=(11,5))
pylab.title('Comparing K-Means Centers with Data Means',fontsize=15)
pylab.xticks(rotation=0,fontsize=10); pylab.grid(); pylab.show()

In [ ]:
log_centers2=pca.inverse_transform(centers2)
true_centers2=numpy.exp(log_centers2)
segments2=['Segment {}'.format(i) for i in range(0,len(centers2))]
true_centers2=pandas.DataFrame(numpy.round(true_centers2),
                               columns=data.keys(),index=segments2)
print ("Gaussian Mixture Model True Centers"); true_centers2

In [ ]:
true_centers2=true_centers2.append(data.describe().loc['mean'])
true_centers2.plot(kind='bar',figsize=(11,5))
st='Comparing Gaussian Mixture Model Centers with Data Means'
pylab.title(st,fontsize=15)
pylab.xticks(rotation=0,fontsize=10); pylab.grid(); pylab.show()

In [ ]:
clusters=pandas.DataFrame(2*[['','']],
                          columns=['K-Means','Gaussian Mixture'],
                          index=['Segment 0','Segment 1'])
if true_centers['Fresh']['Segment 0']>true_centers['Fresh']['Segment 1']:
    clusters['K-Means']['Segment 0']='HoReCa'
    clusters['K-Means']['Segment 1']='Retail'
else: 
    clusters['K-Means']['Segment 0']='Retail'
    clusters['K-Means']['Segment 1']='HoReCa'
if true_centers2['Fresh']['Segment 0']>true_centers2['Fresh']['Segment 1']:
    clusters['Gaussian Mixture']['Segment 0']='HoReCa'
    clusters['Gaussian Mixture']['Segment 1']='Retail'
else: 
    clusters['Gaussian Mixture']['Segment 0']='Retail'
    clusters['Gaussian Mixture']['Segment 1']='HoReCa'
clusters

In [ ]:
clusters2=pandas.DataFrame(3*[['','']],
                           columns=['K-Means','Gaussian Mixture'],
                           index=['C0','C1','C2'])
clusters2['Cluster']=['Retail','Retail','HoReCa']
for el in ['K-Means','Gaussian Mixture']:
    if clusters[el].values[0]=='Retail': 
        clusters2[el]=['Segment 0','Segment 0','Segment 1']
    else: 
        clusters2[el]=['Segment 1','Segment 1','Segment 0']
clusters2

In [ ]:
st1="Sample point {} predicted to be in Cluster {}"
st2="The distance between sample point {} and center of Cluster {}:"
print ("K-Means")
for i,pred in enumerate(sample_preds):
    print (st1.format(i,pred))
    print (st2.format(i,pred))
    print (samples.iloc[i]-true_centers.iloc[pred])
print ("\n\nGaussian Mixture Model")
for i,pred in enumerate(sample_preds2):
    print (st1.format(i,pred))
    print (st2.format(i,pred))
    print (samples.iloc[i]-true_centers2.iloc[pred])

In [ ]:
channel_results(reduced_data,outliers,pca_samples[:3])

In [ ]:
channel_data=pandas.read_csv(path+file)
channel_data=channel_data.drop(
    channel_data.index[outliers]).reset_index(drop=True)
if clusters2['K-Means'].values[0]=='Segment 0': v11,v12=0,1
else: v11,v12=1,0
if clusters2['Gaussian Mixture'].values[0]=='Segment 0': v21,v22=0,1
else: v21,v22=1,0
df=numpy.where(channel_data['Channel']==2,v11,v12)
percentage_kmeans=sum(df==preds)/float(len(preds))
df2=numpy.where(channel_data['Channel']==2,v21,v22)
percentage_gaussian_mixture=sum(df2==preds2)/float(len(preds2))
print ('K-Means')
print ('Percentage of correctly classified customers: {:.2%}'\
.format(percentage_kmeans))
print ('\nGaussian Mixture Model')
print ('Percentage of correctly classified customers: {:.2%}'\
.format(percentage_gaussian_mixture))